# Proyecto 1

# Importaciones

In [25]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from sklearn import linear_model
from joblib import dump, load

from sklearn.pipeline import Pipeline

from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Carga de datos

In [26]:
data=pd.read_csv('SuicidiosProyecto.csv', sep=',', encoding = 'utf-8')
data_s=data

# Entendimiento de los datos

In [ ]:
data_s.head()

In [ ]:
data_s.dtypes

In [ ]:
data_s.describe()

In [ ]:
data_s.shape

In [ ]:
data_s["class"].value_counts()

# Limpieza de los datos

In [27]:
data_s["class"] = data_s["class"].replace({'suicide':1, 'non-suicide':0})

In [28]:
X, y = data_s['text'], data_s['class']

In [29]:
documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split(' ')

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    X[sen]=document



C:\Users\Windows\AppData\Local\Temp\ipykernel_17272\3929755075.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[sen]=document


In [ ]:
tfidfconverter = TfidfVectorizer()
X = tfidfconverter.fit_transform(X).toarray()

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Implementación algoritmos 

## MultinomialNB


In [ ]:
m = MultinomialNB()
m.fit(X_train, y_train)

In [ ]:
y_pred= m.predict(X_test)

print('Exactitud: %.2f' % accuracy_score(y_test, y_pred))
print("Recall: {}".format(recall_score(y_test,y_pred)))
print("Precisión: {}".format(precision_score(y_test,y_pred)))
print("Puntuación F1: {}".format(f1_score(y_test,y_pred)))

In [ ]:
print(classification_report(y_test, y_pred))

## GaussianNB


In [ ]:
g = GaussianNB()
g.fit(X_train, y_train)

In [ ]:
y_pred= g.predict(X_test)

print('Exactitud: %.2f' % accuracy_score(y_test, y_pred))
print("Recall: {}".format(recall_score(y_test,y_pred)))
print("Precisión: {}".format(precision_score(y_test,y_pred)))
print("Puntuación F1: {}".format(f1_score(y_test,y_pred)))

In [ ]:
print(classification_report(y_test, y_pred))

## Regresión Logística


In [ ]:
model = linear_model.LogisticRegression()
model.fit(X_train,y_train)

In [ ]:
predictions = model.predict(X_test)
#Cuan bueno fue nuestro modelo
model.score(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)
print('Exactitud: %.2f' % accuracy_score(y_test, y_pred))
print("Recall: {}".format(recall_score(y_test,y_pred)))
print("Precisión: {}".format(precision_score(y_test,y_pred)))
print("Puntuación F1: {}".format(f1_score(y_test,y_pred)))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)
plot_confusion_matrix(model, X_test, y_test)  
plt.show()  

# Creación de Pipeline

In [31]:
def Limpieza(text):
    documents = []
    stemmer = WordNetLemmatizer()

    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(text))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split(' ')

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    return document

In [32]:
model = ('model', linear_model.LogisticRegression())
pipeline = Pipeline(steps=[('data_preprocessing', TfidfVectorizer(lowercase=False)), ('model', linear_model.LogisticRegression())])

In [33]:
pipeline = pipeline.fit(X_train,y_train)

c:\Users\Windows\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
pipeline.score(X_test,y_test)

0.9369187531936638

In [ ]:

rta= Limpieza("General Kenobi, Hello There First one to comment does not get an award because I've never gotten one even though everyone's saying I should be getting one every week or so")

In [24]:
filename = 'model.joblib'
dump(pipeline, filename) 

['vectorizador.joblib']

# Prediccion datos sin etiqueta

## Carga de datos sin etiqueta

In [14]:
data_SE=pd.read_csv('SuicidiosProyectoSinEtiqueta.csv', sep=',', encoding = 'utf-8')
data_SE_P=data_SE

In [ ]:
data_SE_P.head()

## Limpieza de datos sin etiqueta

In [15]:
XP = data_SE_P['text']

In [ ]:
print(XP)

## Prediccion

In [16]:
data_SE_P['class'] = pipeline.predict(XP)
data_SE_P["class"] = data_SE_P["class"].replace({1:'suicide', 0:'non-suicide'})

In [17]:
data_SE_P.head()

,Unnamed: 0,text,class
0,66529,"Suicide Thoughts Getting StrongerOK, since the...",suicide
1,293637,"Guys, I just asked my crush... if she wanted t...",non-suicide
2,198884,I'm going to kill myselfOf course I'm not but ...,suicide
3,186897,I did it! I slept facing the wall.\n\nOverall ...,non-suicide
4,89024,I don't want to be dead...I want to be alive a...,suicide


In [18]:
data_SE_P["class"].value_counts()

non-suicide    5896
suicide        4404
Name: class, dtype: int64